# Hyperbolic trajectory
----

Import modules and functions

In [2]:
import numpy as np
from numpy import cos, sin, cosh, sinh, tanh, array,pi, exp
from numpy.linalg import norm,solve
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import scipy.linalg
from System import MakeSystem, FindHyperbolicTrajectory, MakeLinearisedSolution
%matplotlib qt

Set parameters of the system and find the hyperbolic trajectory, as well as the linearised hyperbolic trajectory

In [3]:
k = 3
ForcingParameters = array([.1,.7,-.15,2])
f,lminus,lplus,G,A = MakeSystem(k,ForcingParameters) 

XQ = MakeLinearisedSolution(ForcingParameters,lplus,lminus)
tmin,tmax = -10,10

N = 401
Xhyp,t,X0 = FindHyperbolicTrajectory(f,lminus,lplus,XQ,tmin,tmax,N,A)

 after 7 iterations an estimate with error of F =2.4505287786777723e-11 and delta = 5.391677075353767e-09was produced


Produce plots comparing the hyperbolic trajectory with the linearised system.

In [ ]:
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(projection='3d')

ax.plot(Xhyp[:,0], Xhyp[:,1], t,linestyle='-',  linewidth=3, marker='o', label="Non-linear hyperbolic trajectory, quasi-periodic")
ax.plot( X0[:,0], X0[:,1],t,linestyle='--', linewidth=2, marker='x', label="Linearised hyperbolic trajectory, quasi-periodic")
plt.legend(fontsize=14)
ax.set_xlabel("$x$", fontsize=14)
plt.xlim(-1, 1)
plt.ylim(-1, 1)
ax.set_ylabel("$y$", fontsize=14)
ax.set_zlabel("$t$", fontsize=14)
plt.show()

plt.plot(t, Xhyp[:,0], color='blue', linestyle='-',  linewidth=3, marker='o', label="Non-linear hyperbolic trajectory")
plt.plot(t, X0[:,0], color='red',  linestyle='--', linewidth=2, marker='x', label="Linearised hyperbolic trajectory")
plt.title("Hyperbolic trajectories $x$ component, quasi-periodic", fontsize=20)
plt.xlabel("$t$", fontsize=18)
plt.ylabel("$x$", fontsize=18)
plt.legend(fontsize=6)
plt.show()
plt.plot(t, Xhyp[:,1], color='blue', linestyle='-',  linewidth=3, marker='o', label="Non-linear hyperbolic trajectory")
plt.plot(t, X0[:,1], color='red',  linestyle='--', linewidth=2, marker='x', label="Linearised hyperbolic trajectory")
plt.title("Hyperbolic trajectories $y$ component, quasi-periodic", fontsize=20)
plt.xlabel("$t$", fontsize=18)
plt.ylabel("$y$", fontsize=18)
plt.legend(fontsize=6)
plt.show()

# Stable manifold data collection
---

Import functions to collect and process stable manifold data

In [4]:
from StableManifoldTrajectories import FindStableTrajectory,FindUnstableTrajectory
import scipy.io
import scipy.interpolate
from scipy.interpolate import RegularGridInterpolator
from scipy.interpolate import Rbf
from scipy.interpolate import LinearNDInterpolator
from scipy.interpolate import interp1d


## ONLY RUN ONCE
This code collects points on the stable manifold. This takes the longest to run, so only run if no available data for the model and forcing parameters.

In [5]:
stableCoord = np.linspace(-2,2,15)
unstableCoord =np.linspace(-2,2,15)
tindices = np.arange(int(N//5),int(4*N//5),25)
ySMvals = np.zeros((len(stableCoord),len(tindices)))
xSMvals = np.zeros((len(stableCoord),len(tindices)))
yUMvals = np.zeros((len(unstableCoord),len(tindices)))
xUMvals = np.zeros((len(unstableCoord),len(tindices)))


count =0
l=0
for J in tindices:
    for i in range(0,len(stableCoord)):
        p = stableCoord[i]
        X,tnew,X0,Jnew = FindStableTrajectory(f,lminus,lplus,tmin,tmax,N,p,J,Xhyp,A)
        ySMvals[i,l] = X[Jnew][1]   # output of function applied to (trange[J],P[i])
        xSMvals[i,l] = X[Jnew][0]   # output of function applied to (trange[J],P[i])
        count +=1 
        if int(100*count/(len(tindices)*len(stableCoord)))%5 ==0:
            print(str(int(100*count/(len(tindices)*len(stableCoord)))) + "% complete")
    l+=1
            
count =0
l=0
for J in tindices:
    for i in range(0,len(unstableCoord)):
        q = unstableCoord[i]
        X,tnew,X0,Jnew = FindUnstableTrajectory(f,lminus,lplus,tmin,tmax,N,q,J,Xhyp,A)
        yUMvals[i,l] = X[Jnew][1]   # output of function applied to (trange[J],P[i])
        xUMvals[i,l] = X[Jnew][0]   # output of function applied to (trange[J],P[i])
        count +=1 
        if int(100*count/(len(tindices)*len(unstableCoord)))%5 ==0:
            print(str(int(100*count/(len(tindices)*len(unstableCoord)))) + "% complete")
    l+=1
            
            
            

0% complete
5% complete
10% complete
10% complete
15% complete
20% complete
20% complete
25% complete
30% complete
30% complete
35% complete
40% complete
40% complete
45% complete
50% complete
50% complete
55% complete
60% complete
60% complete
65% complete
70% complete
70% complete
75% complete
80% complete
80% complete
85% complete
90% complete
90% complete
95% complete
100% complete
0% complete
5% complete
10% complete
10% complete
15% complete
20% complete
20% complete
25% complete
30% complete
30% complete
35% complete
40% complete
40% complete
45% complete
50% complete
50% complete
55% complete
60% complete
60% complete
65% complete
70% complete
70% complete
75% complete
80% complete
80% complete
85% complete
90% complete
90% complete
95% complete
100% complete


xSM[i,j] stores the stable manifold x position corresponding to (stableCoord[i],t[tindices[l]])

In [6]:
np.save("tindices.npy",tindices)
np.save("t.npy",t)
np.save("Xhyp.npy",Xhyp)
np.save("k.npy", k)
np.save("ForcingParameters.npy",ForcingParameters)

np.save("stableCoord.npy",stableCoord)
np.save("ySMvals.npy",ySMvals)
np.save("xSMvals.npy",xSMvals)


np.save("unstableCoord.npy",unstableCoord)
np.save("yUMvals.npy",yUMvals)
np.save("xUMvals.npy",xUMvals)



## Importing already computed data


In [3]:
tindices=np.load("tindices.npy")
t=np.load("t.npy")
Xhyp=np.load("Xhyp.npy")
k = np.load("k.npy")
ForcingParameters= np.load("ForcingParameters.npy")

stableCoord = np.load("stableCoord.npy")

ySMvals= np.load("ySMvals.npy")
xSMvals = np.load("xSMvals.npy")


unstableCoord = np.load("unstableCoord.npy")
yUMvals = np.load("yUMvals.npy")
xUMvals = np.load("xUMvals.npy")
f,lminus,lplus,G,A = MakeSystem(k,ForcingParameters) # system in stable/unstable coordinates


Here we create the parameterisations for the stable and unstable manifolds.

In [7]:
NewstableCoord,NewT = np.meshgrid(stableCoord,t[tindices], indexing='ij')

ySM = scipy.interpolate.LinearNDInterpolator(np.vstack((NewstableCoord.reshape(-1),NewT.reshape(-1))).transpose(),ySMvals.reshape(-1))
xSM = scipy.interpolate.LinearNDInterpolator(np.vstack((NewstableCoord.reshape(-1),NewT.reshape(-1))).transpose(),xSMvals.reshape(-1))

NewUnstableCoord,NewT = np.meshgrid(unstableCoord,t[tindices], indexing='ij')

yUM = scipy.interpolate.LinearNDInterpolator(np.vstack((NewUnstableCoord.reshape(-1),NewT.reshape(-1))).transpose(),yUMvals.reshape(-1))
xUM = scipy.interpolate.LinearNDInterpolator(np.vstack((NewUnstableCoord.reshape(-1),NewT.reshape(-1))).transpose(),xUMvals.reshape(-1))

## Plots
----

This plot shows the hyperbolic trajectory alongside the stable and unstable manifolds and sample transition and non-transition trajectories.

In [14]:
fig = plt.figure(figsize=(7,9))
ax = fig.add_subplot(projection='3d')
ax.plot(Xhyp[:,0], Xhyp[:,1], t,linestyle='-',  linewidth=3, marker='x',color = 'green', label="Hyperbolic trajectory")
stableCoordinterp = np.linspace(stableCoord[0]+0.1,stableCoord[-1]-0.1,15)
unstableCoordinterp = np.linspace(unstableCoord[0]+0.1,unstableCoord[-1]-0.1,15)
tinterp = np.linspace(-6, 6, 10)
FontSize = 18
XS = 0*np.zeros((len(stableCoordinterp),len(tinterp)))
YS = 0*np.zeros((len(stableCoordinterp),len(tinterp)))
tS = 0*np.zeros((len(stableCoordinterp),len(tinterp)))
for i in range(0,len(stableCoordinterp)):
    for j in range(0,len(tinterp)):
        XS[i,j] = xSM(stableCoordinterp[i],tinterp[j])
        YS[i,j] = ySM(stableCoordinterp[i],tinterp[j])
        tS[i,j] = tinterp[j]
        
XU = 0*np.zeros((len(unstableCoordinterp),len(tinterp)))
YU = 0*np.zeros((len(unstableCoordinterp),len(tinterp)))
tU = 0*np.zeros((len(unstableCoordinterp),len(tinterp)))
for i in range(0,len(unstableCoordinterp)):
    for j in range(0,len(tinterp)):
        XU[i,j] = xUM(unstableCoordinterp[i],tinterp[j])
        YU[i,j] = yUM(unstableCoordinterp[i],tinterp[j])
        tU[i,j] = tinterp[j]
ax.plot_wireframe(XU, YU, tU, color = 'red',label = 'Unstable manifold',alpha = 0.5)
ax.plot_wireframe(XS, YS, tS, color = 'blue',label = 'Stable manifold',alpha = 0.5)
y0 = np.array([-2,3])
res = solve_ivp (f, [-3,3], y0 , method = "RK45" ,max_step =0.05 , rtol = 1e-8 )
y1 = res .y. transpose()
t1 = res .t  
ax.plot(y1[:,0],y1[:,1],t1, color = 'black', linewidth=3,label = 'Transition trajectory')

y0 = np.array([-2,1])
res = solve_ivp (f, [-3,3], y0 , method = "RK45" ,max_step =0.05 , rtol = 1e-8 )
y2 = res .y. transpose()
t2 = res .t  
ax.plot(y2[:,0],y2[:,1],t2, color = 'brown', linewidth=3,label = 'Non-transition trajectory')

ax.set_xlabel("$x$", fontsize=FontSize)
plt.legend(fontsize=FontSize,bbox_to_anchor=(-0, 1.2), loc='upper left')
ax.set_ylabel("$y$", fontsize=FontSize)
ax.set_zlabel("$t$", fontsize=FontSize)
ax.zaxis.set_tick_params(labelsize=14)
ax.xaxis.set_tick_params(labelsize=14)
ax.yaxis.set_tick_params(labelsize=14)
plt.show()

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

Time slice in original coordinates.

In [7]:
N = len(t)
fig = plt.figure(figsize=(15,15))
J =N//2
fig = plt.figure(figsize=(15,15))
FontSize = 20
plt.plot(Xhyp[J,0],Xhyp[J,0], linewidth=3, marker='o', label="Non-linear hyperbolic trajectory, quasi-periodic")
stableCoordinterp = np.linspace(stableCoord[0]+0.1,stableCoord[-1]-0.1,15)
unstableCoordinterp = np.linspace(unstableCoord[0]+0.1,unstableCoord[-1]-0.1,15)
YSvals =0*stableCoordinterp
XSvals =0*stableCoordinterp
for i in range(0,len(stableCoordinterp)):
    YSvals[i] = ySM(stableCoordinterp[i],t[J])
    XSvals[i] =xSM(stableCoordinterp[i],t[J])
plt.plot(XSvals,YSvals, color = 'blue', linewidth=3, marker='x',label = 'Stable manifold')
YUvals =0*unstableCoordinterp
XUvals =0*unstableCoordinterp
for i in range(0,len(unstableCoordinterp)):
    YUvals[i] = yUM(unstableCoordinterp[i],t[J])
    XUvals[i] =xUM(unstableCoordinterp[i],t[J])
plt.plot(XUvals,YUvals,color = 'red', linewidth=3, marker='x',label = 'Unstable manifold')
plt.legend(fontsize=FontSize)
plt.xlabel("$x$", fontsize=FontSize)
    #plt.xlim(-1, 1)
    #plt.ylim(-1, 1)
plt.ylabel("$y$", fontsize=FontSize)
plt.show()

# Heat map
----


This is an interpolation of the hyperbolic trajectory.

In [9]:
xhyp = lambda s: np.interp(s,t,Xhyp[:,0])
yhyp = lambda s: np.interp(s,t,Xhyp[:,1])

Load saved data

In [10]:
x = np.load("initial conditions.npy")
z = np.load("Points moving over.npy")

Construct new data
# Do not run if data already available!


In [10]:
def event(s,y):
    #return (Minv.dot(y) - np.array([xhyp(s),yhyp(s)]))[0]
    return y[0]- xhyp(s)
event.terminal = True
# functions to determine if it passes over
def passOver(y0):
    if event(0,y0) < 0:
        res = solve_ivp (f, [0,100], y0 , method = "RK45" , events=[event],max_step =0.05 , rtol = 1e-8 )
        y = res .y. transpose()
        t = res .t  
        s = t[-1]
        if s<100 and y[-1][1] > yhyp(s):  # the trajectories which move left to right
            return s
        elif y[-1][0]- xhyp(s) < 0: # trajectories which stay on left
            return -1
    else: 
        res = solve_ivp (f, [0,100], y0 , method = "RK45" , events=[event],max_step =0.05 , rtol = 1e-8 )
        y = res .y. transpose()
        t = res .t  
        s = t[-1]
        if s<100 and y[-1][1] < yhyp(s):  # the trajectories which move right to left 
            return -1
        elif y[-1][0]- xhyp(s) > 0: # trajectories which stay on right
            return 0



#creating and storing the data.

x = np.meshgrid(np.linspace(-3,xhyp(0)+.5,150) , np.linspace(yhyp(0)-0.5,3,150),indexing= 'ij')
z = np.zeros((np.shape(x)[1],np.shape(x)[2]))
count = 0
for i in range(0,np.shape(x)[1]):
    for j in range(0,np.shape(x)[2]):
        y0 = np.array([x[0][i,j],x[1][i,j]])
        z[i,j] = passOver(y0)
        count+=1
        if int(100000*count/(np.shape(x)[1]*np.shape(x)[2])) % 5 == 0:
            print(str(100*count/(np.shape(x)[1]*np.shape(x)[2])) + "% complete")
np.save("initial conditions.npy" , x)
np.save("Points moving over.npy", z)

0.035555555555555556% complete
0.04% complete
0.07555555555555556% complete
0.08% complete
0.11555555555555555% complete
0.12% complete
0.15555555555555556% complete
0.16% complete
0.19555555555555557% complete
0.2% complete
0.23555555555555555% complete
0.24% complete
0.27555555555555555% complete
0.28% complete
0.31555555555555553% complete
0.32% complete
0.35555555555555557% complete
0.36% complete
0.39555555555555555% complete
0.4% complete
0.43555555555555553% complete
0.44% complete
0.47555555555555556% complete
0.48% complete
0.5155555555555555% complete
0.52% complete
0.5555555555555556% complete
0.56% complete
0.5955555555555555% complete
0.6% complete
0.6355555555555555% complete
0.64% complete
0.6755555555555556% complete
0.68% complete
0.7155555555555555% complete
0.72% complete
0.7555555555555555% complete
0.76% complete
0.7955555555555556% complete
0.8% complete
0.8355555555555556% complete
0.84% complete
0.8755555555555555% complete
0.88% complete
0.9155555555555556% com

Produce plots, heat map of which points transition and time to transition.

In [13]:
X,Y = x
YSvals =0*stableCoordinterp
XSvals =0*stableCoordinterp
for i in range(0,len(stableCoordinterp)):
    YSvals[i] = ySM(stableCoordinterp[i],0)
    XSvals[i] =xSM(stableCoordinterp[i],0)
YUvals =0*unstableCoordinterp
XUvals =0*unstableCoordinterp
for i in range(0,len(unstableCoordinterp)):
    YUvals[i] = yUM(unstableCoordinterp[i],0)
    XUvals[i] =xUM(unstableCoordinterp[i],0)

fig = plt.figure(figsize=(8,8))
plt.xlabel("$x$", fontsize=18)
    #plt.xlim(-1, 1)
    #plt.ylim(-1, 1)
plt.ylabel("$y$", fontsize=18)
import matplotlib.colors
custom_cmap = matplotlib.colors.LinearSegmentedColormap.from_list("custom",["red","orange","yellow","green"])
plt.imshow(z.transpose(), vmin = -1,vmax = 3, cmap = custom_cmap, origin = 'lower',alpha = .8,extent = [np.amin(X),np.amax(X),np.amin(Y),np.amax(Y)])

plt.plot(XSvals[0:XSvals.shape[0]//2+1],YSvals[0:XSvals.shape[0]//2+1], color = 'blue', linewidth=3, marker='x',label = 'Stable manifold')
plt.plot(xhyp(0),yhyp(0), linewidth=3, marker='o', color = 'black', label="Hyperbolic trajectory")
plt.legend(fontsize=18)
plt.colorbar()
plt.ylim(np.amin(Y),np.amax(Y))
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.show()

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [12]:
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(projection='3d')
X,Y = x
ax.plot_surface(X,Y,z)
plt.show()